In [1]:

import os
import pandas as pd
import json

from model import num2cate_fit, num2cate_transform, generate_samples, generate_model_samples, ModelGene, findKeyAttrs, FindGroups, get_numAttrs, find_rules
from model.samples import DataGene
from model.data_encoder import DataEncoder
from joblib import dump, load


In [17]:


store_path = '../../front/src/testdata/'


def get_rules(dataset_name, protect_attr='',model_name=None):
    
    model_name = '{}_{}'.format(dataset_name, model_name)
    sample_path = os.path.join(store_path, '{}_samples.csv'.format(model_name))
    model_samples = pd.read_csv(sample_path)
    model_samples = model_samples.iloc[int(len(model_samples)/2):]
    rules = find_rules(model_samples, minimum_support=5, min_len=1, protect_attr = protect_attr, target_attr='class', elift_th=[1, 1])

    
    rules.to_json(store_path + '{}_rules.json'.format(model_name),orient='records')
    
    
def init_samples(dataset_name):
    sample_num = 1000 # number of generated data 
    dataset_path = '../server/{}.csv'.format(dataset_name)
    data = pd.read_csv(dataset_path)
    
    
    # generate samples
    samplesInit = generate_samples(data, sample_num)
    samples_path = os.path.join(store_path, '{}_samples.json'.format(dataset_name))
    samplesInit.to_json(samples_path, orient='records')


def get_model_samples(dataset_name, models=['lr', 'knn', 'xgb'], protect_attr=''):
    """
    train model on the training data,
    return the generated samples based on the training data
    """
    
    dataset_path = '../server/{}.csv'.format(dataset_name)
    data = pd.read_csv(dataset_path)
    mdlp = num2cate_fit(data)

    samples_path = os.path.join(store_path, '{}_samples.json'.format(dataset_name))
    samplesInit = pd.read_json(samples_path)
    
    

#     # find key_attrs
#     key_attrs = findKeyAttrs(num2cate_transform(data, mdlp), protect_attr)
#     f = open(store_path + '{}_key.json'.format(dataset_name),'w')
#     json.dump(key_attrs, f)
    
    for var in models:
        
        # set name
        model_name = dataset_name+"_"+var
        model_gene = ModelGene(model_name)
    
        # train model
        model, encoder, score = model_gene.fit_model( num2cate_transform(data, mdlp) )
        
        # general samples
    
        num_samples, cate_samples = generate_model_samples(samplesInit, mdlp, model, encoder) 
        
        # add the ID col 
        cate_samples.insert(loc=0, column='id', value= cate_samples.index)
        num_samples.insert(loc=0, column='id', value= num_samples.index)
        
        # save mdeol & samples to cache
        dataOut = pd.concat([num_samples,cate_samples])
        
        samples_path = os.path.join(store_path, '{}_samples.json'.format(model_name))
        dataOut.to_json(samples_path, orient='records')

        samples_path = os.path.join(store_path, '{}_samples.csv'.format(model_name))
        dataOut.to_csv(samples_path, index=False)
        
        print(dataset_name + ' ' + var + ' accuracy: ' + str(score))
        
    print(dataset_name + ' all done')



In [3]:
# init_samples('simpleAcademic')
get_model_samples('academic', ['dt'])

/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/myenv/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/myenv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/wangqianwen/Documents/git_repo/disc

academic dt accuracy: 0.9166666666666666
academic all done


In [ ]:
# get_rules('academic','gender=F', 'xgb')

def get_all_rules(protect, models=['xgb','knn','lr'], dataset='adult'):
    
    for model in models:
        get_rules(dataset, protect, model)
        get_rules(dataset, protect, model)
        get_rules(dataset, protect, model)
    for model in models:
        sample_name = os.path.join(store_path, dataset + '_' + model + '_samples.json')
        rule_name = os.path.join(store_path, dataset + '_' + model + '_rules.json')
        samples = pd.read_json(sample_name)

        # add item id to rules
        def item_within_rule(item, rule_context):
            for attr_val in rule_context:
                attr, val = attr_val.split('=')
                if not item[attr] == val :
                    return False
            return True

        rules = pd.read_json(rule_name)
        rules['items'] = ''
        for idx, rule in rules.iterrows():
            rules.at[idx, 'items'] = [sample['id'] for i,sample in samples.iloc[int(len(samples)/2):].iterrows() if item_within_rule(sample, rule["antecedent"])] 
        rules.to_json(rule_name, orient='records')

        print(dataset+'_'+model+' has done')

    print('All finished')



In [ ]:
get_all_rules('gender=F', ['knn_post'],'academic')

In [ ]:
import pandas as pd


df = pd.read_csv('german_credit.csv')

df['gender'] = ''
df['status'] = ''

for i,d in df.iterrows():
    df.at[i,'gender'] = df.at[i,'personal_status'].split(' ')[0]
    df.at[i,'status'] = df.at[i,'personal_status'].split(' ')[1]
df.drop(['personal_status'],axis=1)
df.to_csv('german_credit.csv',index=False)

In [ ]:
'''explore the number of samples for each rules'''
import pandas as pd

df = pd.read_json('academic_knn_rules.json')
df = pd.DataFrame(df)
for i,d in df.iterrows():
    print(df.at[i,'antecedent'])
    if(df.at[i,'antecedent']==['StudentAbsenceDays=Under-7', 'raisedhands=0<x<25', 'Discussion=1<x<25']):
        print(df.at[i,'id'])

In [16]:
samples = pd.read_json(os.path.join(store_path, '{}_samples.json'.format('academic')))
samples.iloc[int(len(samples)/2):]

,AnnouncementsView,Discussion,Grade,Relation,Semester,StudentAbsenceDays,VisITedResources,gender,raisedhands
500,88,86,7,Mum,F,Under-7,62,F,92
501,30,16,4,Mum,F,Under-7,87,M,1
502,55,24,4,Mum,S,Under-7,5,M,15
503,70,10,7,Father,S,Above-7,8,M,97
504,31,29,9,Father,S,Above-7,64,M,27
505,24,44,6,Father,S,Above-7,50,M,82
506,57,31,5,Father,F,Above-7,70,F,23
507,13,40,10,Mum,F,Under-7,27,M,51
508,71,61,6,Mum,S,Under-7,18,F,1
509,80,80,7,Father,F,Above-7,2,M,97
